## Read in the data

In [ ]:
from __future__ import print_function, division
from collections import defaultdict, Counter
from tqdm import tqdm
from aer import read_naacl_alignments, AERSufficientStatistics
from random import random
from scipy import special
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import os

In [ ]:
english_train = 'training/hansards.36.2.e'
french_train = 'training/hansards.36.2.f'
english_val = 'validation/dev.e'
french_val = 'validation/dev.f'
fname = 'naacltest.txt'

def add_unk(original_sentences, threshold, ext_sentences=None):
    #use original sentences and unk low frequency words
    if ext_sentences is None: 
        counts = Counter([item for sublist in original_sentences for item in sublist])
        unk_words = list({k:counts[k] for k in counts if counts[k] <= threshold})
    #use external sentences and unk unknown words
    else:
        counts = Counter([item for sublist in ext_sentences for item in sublist])
        unk_words = list({k:counts[k] for k in counts})
    for line, sentence in enumerate(tqdm(original_sentences)):
        for index, word in enumerate(sentence):
            if word in unk_words and ext_sentences is None:
                original_sentences[line][index] = '-UNK-'
            elif word not in unk_words and ext_sentences is not None:
                original_sentences[line][index] = '-UNK-'
    return original_sentences
    

def read_data(english_file, french_file, unk=False, threshold=1, ttype='training', eng_data=None, fre_data=None):
    english_sentences = []
    french_sentences = []
    with open(english_file, 'r', encoding='utf8') as engf, open(french_file, 'r', encoding='utf8') as fref:
        for line in engf:
            english_sentences.append(["NULL"] + line.split())
        for line in fref:
            french_sentences.append(line.split())
    
    #unk cases
    if unk:
        print("Adding the -UNK- token to the data.")
        englishname = ttype +'_'+ str(threshold)+'_unk.e'
        frenchname = ttype +'_'+ str(threshold)+'_unk.f'
        #load if file is found
        if os.path.isfile(englishname):
            with open (englishname, 'rb') as eng:
                english_sentences = pickle.load(eng)
        else:
            english_sentences = add_unk(english_sentences, threshold, eng_data)
            with open(englishname, 'wb') as eng:
                pickle.dump(english_sentences, eng)
        print("English data complete.")
        
        
        if os.path.isfile(frenchname):
            with open(frenchname, 'rb') as fre:
                french_sentences = pickle.load(fre)
        else:
            french_sentences = add_unk(french_sentences, threshold, fre_data)
            with open(frenchname, 'wb') as fre:
                pickle.dump(french_sentences, fre)             
        print("French data complete")          
        
    assert len(english_sentences) == len(french_sentences), 'data mismatch'
    return list(zip(english_sentences, french_sentences))

training_data = read_data(english_train, french_train, True)
ext_data = list(zip(*training_data))
validation_data = read_data(english_val, french_val, True, ttype='validation', eng_data=ext_data[0], fre_data=ext_data[1])

# AER Cell

In [ ]:
def test(path, personal_sets=None):
    from random import random
    # 1. Read in gold alignments
    gold_sets = read_naacl_alignments('validation/dev.wa.nonullalign')

    # 2. Here you would have the predictions of your own algorithm
    if personal_sets is None:
        personal_sets = read_naacl_alignments(path)
        predictions = []
        for s, p in personal_sets:
            links = set()
            for link in s:
                links.add(link)
            predictions.append(links)
    else:
        predictions=personal_sets

    # 3. Compute AER
    # first we get an object that manages sufficient statistics 
    metric = AERSufficientStatistics()
    # then we iterate over the corpus 
    for gold, pred in zip(gold_sets, predictions):
        metric.update(sure=gold[0], probable=gold[1], predicted=pred)
    # AER
    return metric.aer()

#hardcoded AER test for 5 iterations
# for i in range(10):
#     test('iteration'+str(i)+'.txt')

# IBM 2

In [ ]:
def align_all(data, translate_dict, jump_dict, fname=None):
    """Create alignments for pairs of English and French sentences.
    Both save them as sets per sentence and pair and save to file.
    
    Args:
        validation: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f
        fname: filename to save alignments in, in NAACL format

    Returns:
        list of sets
    """
    file = open(fname, 'w')
    alignments = []
    for k, (english_words, french_words) in enumerate(data):
        alignment = align(english_words, french_words, translate_dict, jump_dict, False)
        for pos1, pos2 in alignment:
            file.write("{} {} {}\n".format(str(k+1), str(pos1), str(pos2)))
        alignments.append(set(alignment))
    return alignments
    
def align(english_words, french_words, translate_dict, jump_dict, add_null=True):
    """Align one sentence pair, either with or without the NULL alignments.
    
    Args:
        english_words: list of english words
        french_words: list of french words
        translate_dict: dictionary with translation probabilities e to f
        add_null: boolean to indicate whether NULL alignments should be included

    Return:
        list of tuples
    """
    alignment = []
    for j, fword in enumerate(french_words):
        prior = 0.0
        alignment_j = 0
        for i, eword in enumerate(english_words):
            # Only include terms that are in the dictionary
            if eword in translate_dict and fword in translate_dict[eword]:
                prob = translate_dict[eword][fword] * jump_dict[get_jump(j, i, len(english_words), len(french_words))]
                if prob > prior:
                    prior = prob
                    alignment_j = i
        # Add dependent on whether it's a NULL alignments
        if alignment_j != 0 or add_null:
            alignment.append((alignment_j, j + 1))
    return alignment

def initialize_t(data, uniform=True):
    """Initialise the translation probabilities.
    
    Args:
        data: list of tuples, english and french sentences
        uniform: boolean indicating initialisation type

    Returns:
        defaultdict(Counter)
    """
    co_counts = defaultdict(Counter)
    for e, f in data:
        for e_word in e:
            for f_word in f:
                if uniform:
                    co_counts[e_word][f_word] = 1
                else:
                    co_counts[e_word][f_word] = random()
    for e_word in co_counts:
        normalization_factor = sum(list(co_counts[e_word].values()))
        for f_word in co_counts[e_word]:
            co_counts[e_word][f_word] = co_counts[e_word][f_word] / normalization_factor
    return co_counts

def log_likelihood(data, translate_dict, jump_dict):
    """
    Args:
        data: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f

    Returns:
        float: log likelihood
    """
    log_likelihood = 0
    for e, f in data:
        alignment = align(e, f, translate_dict, jump_dict, True)
        logprob = 0
        for i, j in alignment:
            logprob += math.log(translate_dict[e[i]][f[j-1]] * jump_dict[get_jump(j-1, i, len(e), len(f))])
        log_likelihood += logprob
    return log_likelihood

def EM_IBM2(data, validation, initial_translation_estimate=None, initialise_method = 'IBM1', max_steps=3):
    initialise_method = initialise_method.lower()
    assert initialise_method in ['ibm1', 'uniform', 'random'], "initialise method has to be in [ibm1, uniform, random]"
    if initialise_method == 'ibm1':
        assert initial_translation_estimate, "initial_translation_method has to be given, if ibm1"
        translate_dict = initial_translation_estimate
    elif initialise_method == 'uniform':
        translate_dict = initialize_t(data, True)
    else:
        translate_dict = initialize_t(data, False)
    
    jump_dict = {}
    for i in range(-100, 101):
        jump_dict[i] = 1/201
    aers = []
    lls = []

    for iteration in range(max_steps):
        fname = 'IBM2_iteration' + str(iteration) + '.txt'
        counts = defaultdict(int)
        co_counts = defaultdict(int)
        jump_counts = defaultdict(int)
        pos_counts = 0
        
        print("Expectation step {}".format(iteration))
        for e_s,f_s in tqdm(data):
            m = len(f_s)
            l = len(e_s)
            for j, f in enumerate(f_s):
                sum_of_probs = 0
                for i, e in enumerate(e_s):
                    jump_prob = jump_dict.get(get_jump(j,i,l,m), 1/l)
                    translate_prob = translate_dict[e][f]
                    sum_of_probs += jump_prob * translate_prob
                for i, e in enumerate(e_s):
                    jump_prob = jump_dict.get(get_jump(j,i,l,m), 1/l)
                    translate_prob = translate_dict[e][f]
                    prob = jump_prob * translate_prob / sum_of_probs
                    co_counts[(e,f)] += prob
                    counts[e] += prob
                    jump_counts[get_jump(j,i,l,m)] += prob
                    pos_counts += prob

        print("Maximisation step {}".format(iteration))
        for e, f in co_counts:
            translate_dict[e][f] = co_counts[(e, f)] / counts[e]
        for jump in jump_counts:
            jump_dict[jump] = jump_counts[jump] / pos_counts
            
        #writing the iteration files in naacl for AER use
        alignments = align_all(validation, translate_dict, jump_dict, fname)
        ll = log_likelihood(data, translate_dict, jump_dict)
        aer = test("", alignments)
        aers.append(aer)
        lls.append(ll)
        print("AER {}, LL {}".format(aer, ll))
        
        # save models in between
        pickle.dump(translate_dict, open("translate_dicts/ibm2_em_t_epoch_{}.pickle".format(iteration + 1), 'wb'))
        pickle.dump(jump_dict, open("translate_dicts/ibm2_em_jump_epoch_{}.pickle".format(iteration + 1), 'wb'))
    
    # write aers and lls to file
    with open("ibm2_em_scores.txt", 'w') as f:
        f.write("AER")
        for aer in aers:
            f.write("{}\n".format(aer))
        f.write("\nLL")
        for ll in lls:
            f.write("{}\n".format(ll))
    return translate_dict, jump_dict

def get_jump(fre_pos, eng_pos, eng_len, fre_len):
    equivalent_pos = int(math.floor(fre_pos * eng_len / fre_len))
    return eng_pos - equivalent_pos

ibm2_transdict, ibm2_jumpdict = EM_IBM2(training_data[:100], validation_data, initialise_method = 'random', max_steps=20)